In [1]:
from prody import *
from pylab import *
import json

In [2]:
pdb, header = parsePDB('6acg', header=True)

@> PDB file is found in working directory (6acg.pdb).
@> 29715 atoms and 1 coordinate set(s) were parsed in 0.30s.


In [3]:
# get contacts
spike_contacts = pdb.select('chain A B C and within 5 of chain D')
ace2_contacts = pdb.select('chain D and within 5 of chain A B C')

# writePDB('spike_contacts.pdb', spike_contacts)
# writePDB('ace2_contacts.pdb', ace2_contacts)

In [4]:
spike = pdb.select('chain A B C')
ace2  = pdb.select('chain D')

In [5]:
interactions = findNeighbors(atoms=spike, radius=5, atoms2=ace2)

In [6]:
spike_atoms = []
ace2_atoms = []
links = []

In [7]:
# function to format atom data
def getAtomData(atom):
    data = {}

    data["chain"]   = atom.getData('chain')
    data["resname"] = atom.getData('resname')
    data["resnum"]  = int(atom.getData('resnum'))
    data["atom"] = atom.getData('name')
    data["name"]   = " ".join([data["chain"], data['resname'], str(data['resnum']), data['atom']])
    data["id"]   = atom.getIndex()

    return data

In [8]:
# store nodes and links 
for pair in interactions:
    a, b, distance = pair

    spike_atom = getAtomData(a)    
    ace2_atom  = getAtomData(b)

    spike_atoms.append(spike_atom)
    ace2_atoms.append(ace2_atom)

    links.append({
        "source": spike_atom["id"],
        "target": ace2_atom["id"],
        "distance": float(distance)
    })

In [9]:
# remove duplicates in atom lists
spike_atoms = list({v['id']:v for v in spike_atoms}.values())
ace2_atoms  = list({v['id']:v for v in ace2_atoms}.values())

for node in spike_atoms + ace2_atoms:
    counter = 0
    for link in links:
        if link["source"] == node["id"]:
            counter += 1
        if link["target"] == node["id"]:
            counter += 1
    node["links"] = counter

spike_atoms = sorted(spike_atoms, key = (lambda i : i['links']), reverse=True)
ace2_atoms  = sorted(ace2_atoms,  key = (lambda i : i['links']), reverse=False)

In [10]:
interaction_json = {
    "nodes": spike_atoms + ace2_atoms,
    "links": links
}

In [11]:
# write to json
with open('atom_interactions.json', 'w') as file:
    json.dump(interaction_json, file)

In [12]:
# Just for verification
spike_contact_residues  = sorted(spike_contacts.getData('resnum'))
spike_contact_residues2 = sorted([atom['resnum'] for atom in spike_atoms])
spike_contact_residues == spike_contact_residues2

True

In [13]:
spike_residues = []
ace2_residues  = []
residue_links_temp  = []
residue_links = []

for node in spike_atoms + ace2_atoms: 
    residue_node = {
        "chain":   node["chain"],
        "resname": node["resname"],
        "resnum":  node["resnum"],
        "name":    node["chain"] + str(node["resnum"]) + "_" + node["resname"],
        "id":      node["chain"] + str(node["resnum"])
    }
    if node['chain'] == 'C':
        spike_residues.append(residue_node)
    elif node['chain'] == 'D':
        ace2_residues.append(residue_node)

spike_residues = list({v['id']:v for v in spike_residues}.values())
ace2_residues  = list({v['id']:v for v in ace2_residues}.values())    

In [14]:
for link in links:
    source, target = 0, 0
    
    for node in spike_atoms + ace2_atoms:
        if link["source"] == node["id"]:
            source = node["chain"] + str(node["resnum"])
            if target is not 0:
                break
        if link["target"] == node["id"]:
            target = node["chain"] + str(node["resnum"])
            if source is not 0:
                break

    if (source, target) in residue_links_temp:
        for link in residue_links:
            if link['source'] == source and link['target'] == target:
                link['bonds'] += 1
    else:
        residue_links_temp.append((source, target))
        residue_links.append({
            "source": source,
            "target": target,
            "bonds": 1
        })

In [15]:
len(residue_links)

55

In [16]:
for node in spike_residues + ace2_residues:
    counter = 0
    for link in residue_links:
        if link["source"] == node["id"]:
            counter += 1
        if link["target"] == node["id"]:
            counter += 1
    node["links"] = counter

spike_residues = sorted(spike_residues, key = (lambda i : i["links"]), reverse=True)
ace2_residues  = sorted(ace2_residues,  key = (lambda i : i["links"]), reverse=False)


In [17]:
residue_interaction_json = {
    "nodes": spike_residues + ace2_residues,
    "links": residue_links
}

In [18]:
with open('residue_interactions.json', 'w') as file:
    json.dump(residue_interaction_json, file)